# 深層学習のパラメータについて

Kerasにおけるパラメータとしてどんなものがあって、どのような特性を持つのかをまとめておきます。 

## 参考

* [無から始めるKeras](https://qiita.com/Ishotihadus/items/d47fc294ca568536b7f0)
* [ニューラルネットワークのバリエーション](https://thinkit.co.jp/story/2015/09/09/6399)

## サンプルコード

サンプルコードは↓

In [2]:
import tensorflow as tf
X = [0,0,0]
y = [1]
model = tf.keras.models.Sequential()	# Sequentialモデル
# 入力2/出力8/活性化関数tanhの層を追加
model.add(tf.keras.layers.Dense(1024,input_dim=3, activation='tanh'))
# 入力8(省略)/出力1/活性化関数sigmoidの層を追加
model.add(tf.keras.layers.Dense(1, activation='softplus'))
# 学習の仕方を定義
# * 目的関数(ロス関数)としてbinary_crossentropy(logloss)
# * 最適化アルゴリズムにSGD(確率的勾配降下法）、学習率を0.1に設定
# * 評価メトリクスとしてaccuracyを表示
model.compile("adamax", loss='kld',metrics=['accuracy'])

# 活性化関数:Activation

![](https://thinkit.co.jp/sites/default/files/639904.png)

## -1から1の間をとるもの

* softsign
* tanh

tanhのほうが変化が急峻。

## 0以上1以下のみをとるもの

* sigmoid
* hard_sigmoid

ハードシグモイドのほうが急で折れ線。シグモイドは滑らか。
入力が0のとき出力は0.5。上と同様0あたりでは線形に効くが、絶対値が大きくなってくると0か1に張り付く。

## 0以上の値をとるもの

* softplus
* relu

reluほうが急で折れ線。ソフトプラスは（かなり）滑らか。
値がある程度（ゼロから見て）大きくなると線形になり、小さくなるとゼロに近くなる感じ。
reluは係数で0未満の値をとることもできる。

## その他

* 線形関数（linear）
* ソフトマックス（softmax）

線形の方は単純に係数をかけてバイアスを加える（係数とバイアスは学習される？）。ただ活性化関数が線形なのはあまり意味がない気がするのでこれはあまり使わないのかも。

ソフトマックスは、その層から出力されたすべての値に指数関数をかけ（すなわちすべて正にし）、その和を1に正規化する。この関数がかかった後、和は1だしどの値も正なので、確率として判断することができる。

# 目的関数:Objective

compile の2つ目の引数。小さくしたい関数。誤差（loss）とだいたい同じ意味。  
「どんな基準で誤差を決めるか」を決める。  
ここでいう「誤差」とは、今のニューラルネットから推測される値と正解の値の差のことをいう。

種類としては下記がある。
* 平均二乗誤差（mse：差の2乗の和）
* 平均絶対誤差（msa：差の絶対値の和）
* 平均絶対誤差率（mspa：差を正解の値で割った値（誤差率）の絶対値の和）
* 対数平均二乗誤差（msle：「1を加えた値の対数」の差の2乗の和）
* ヒンジ損失の和（hinge）
* ヒンジ損失の二乗の和（squared_hinge）
* 2クラス分類時の交差エントロピー（binary_crossentropy）
* Nクラス分類時の交差エントロピー（categorical_crossentropy）
* スパースなNクラス分類交差エントロピー（sparse_categorical_crossentropy）
* KLダイバージェンス（kld）
* poisson
* コサイン類似度を負にしたもの（cosine_proximity）

# 最適化関数:Optimizer

最適化関数とは、誤差関数の極小値を探索するアルゴリズムです。  
誤差をニューラルネットワークの出力と正解の差とすると、誤差が少なくなるように探索するのに、誤差関数の極小値を探索すればよいことになります。  
基本的な考え方は、例えば3次元の誤差関数とすれば、ボールを置いてどこまで転がるか、で評価できます。(勾配をどれだけ降下するか→勾配降下法)

![Gif](http://1.bp.blogspot.com/-K_X-yud8nj8/VPmIBxwGlsI/AAAAAAAACC0/JS-h1fa09EQ/s1600/Saddle%2BPoint%2B-%2BImgur.gif)

## sgd:確率的勾配降下法

Stochastic Gradient Descentの略で、Optimizerの中でも初期に提唱された最も基本的なアルゴリズムです。  
最も基本になるので、ちょっと解説を書いておきます。

重み$w$の更新は以下のように行っています。このとき、$E$は誤差関数、$\eta$は学習係数を表しています。

${\mathbf{w}^{t + 1} \gets \mathbf{w}^{t} - \eta \frac{\partial E(\mathbf{w}^{t})}{\partial \mathbf{w}^{t}}
}$

この手法は、通常の勾配法で起こりうる局所最適解への収束という問題点を、$w$の更新ごとにランダムにサンプルを選び出すことで解消したものです。  
また、訓練データにおける冗長性を効率良く学習することができるという利点もあります。  
しかし学習係数$\eta$を恣意的に決定する必要があり、一度決めた$\eta$を用いて誤差の最小化を行なっていくことになります。  
このことから、学習モデルによって最適なパラメータを決めることが難しいという問題があります。

## 主だったもの

表に列挙していきます。

|kerasのパラメータ名|アルゴリズム名|日本語訳|パラメータ|特徴|
|-----------------|-----------|---------|-----------|---|
|sgd              |Stochastic Gradient Descent|確率的勾配降下法|$\eta$:学習係数,$\alpha$:慣性項(momentum)||
|adagrad          |AdaGrad                    |?            |初期学習率$\eta$,発散抑制:$\epsilon$|SGDから学習係数を動的に変化するようにしたもの|
|rsmprop          |RSMprop                    |?            |初期学習率$\eta$発散抑制:$\epsilon$,$\alpha$|AdaGradに勾配の2乗の指数移動平均を使って最適化する形に改良|
|adadelta         |AdaDelta                   |?            |$\rho$,発散抑制:$\epsilon$|直近の勾配情報を優先するようAdaGradを改善し初期学習率をパラメータから削除|
|adam             |Adaptive moment estimation |?            |$\alpha$, $\beta_{1}$, $\beta_{2}$, $\epsilon$|移動指数平均により収束率を改善|
|adamax           |?                          |?            |?                                |?|
|nadam            |Nesterov Adam              |?            |?                                |?|

といったものがあります。

## よく使うもの

よくわからないが、よくわからないなりに使うには、$\eta$がなくともある程度収束が速いadamを使うと良さげらしい。  
RNNを使う場合、adamだと収束がだいぶ遅くなるのでrsmpropを使うのが定説らしい。

# ネットワークの種類

大きく分けて教師あり学習/教師なし学習に分類されるが、今回は教師あり学習で使われるものについてまとめておく。

## 全結合

Dense層を使ったもの。  
音声解析等に利用されているらしい。

<img src="https://thinkit.co.jp/sites/default/files/639908.png" width=50%>

## CNN

Convolutional Neural Networkの略。畳み込みニューラルネットワークとも。  
画像解析に利用されている。  
Kerasでは畳み込み層がconv1d/conv2d、プーリング層はMaxPooling1D等を使う。

<img src="https://thinkit.co.jp/sites/default/files/639906.png" width=50%>

## RNN

Recurrent Neural Networkの略。再帰的ニューラルネットとも。  
データの順番を意識して学習してくれるため、時系列データや自然言語処理に利用されている。  
どの程度前の入力を保持するかによってアルゴリズムが多数あるが、LSTM(Long Short-Term Memory)がよく使われているらしい。

<img src="https://thinkit.co.jp/sites/default/files/639907.png" width=50%>
